In [1]:
import pandas as pd

In [2]:
breaches = pd.read_csv("Balloon Race Data Breaches - LATEST - breaches.csv")

In [3]:
breaches['organisation'] = breaches['organisation'].str.lower() 

In [4]:
stocks = pd.read_csv("World-Stock-Prices-Dataset.csv")

In [5]:
stocks['Brand_Name'] = stocks['Brand_Name'].str.lower() 

In [6]:
stocks = stocks.rename(columns={'Brand_Name': 'organisation'})

In [7]:
stocks = stocks.rename(columns={'Date': 'StockDate'})

In [8]:
breaches = breaches.rename(columns={'date': 'BreachDate'}) 

In [9]:
breaches['BreachDate'] = pd.to_datetime(breaches['BreachDate'], format='mixed', utc=True)

In [10]:
breaches['breach_month_year'] = breaches['BreachDate'].dt.strftime('%B %Y')

In [11]:
breaches.dtypes

organisation                      object
alternative name                  object
records lost                      object
year                              object
BreachDate           datetime64[ns, UTC]
story                             object
sector                            object
method                            object
interesting story                 object
data sensitivity                  object
displayed records                 object
Unnamed: 11                      float64
source name                       object
1st source link                   object
2nd source link                   object
ID                               float64
breach_month_year                 object
dtype: object

In [12]:
breaches = breaches.drop(columns=["alternative name","story","interesting story","Unnamed: 11", "source name", "1st source link", "2nd source link", "ID"])

In [13]:
stocks = stocks.drop(columns=["Open","High","Low","Ticker","Dividends","Stock Splits","Capital Gains"])

In [14]:
stocks['StockDate'] = pd.to_datetime(stocks['StockDate'], utc=True)
stocks['stock_month_year'] = stocks['StockDate'].dt.strftime('%B %Y')

In [15]:
stocks.dtypes

StockDate           datetime64[ns, UTC]
Close                           float64
Volume                          float64
organisation                     object
Industry_Tag                     object
Country                          object
stock_month_year                 object
dtype: object

In [18]:
# Extract the year and month from the date column
stocks['year_monthstocks'] = stocks['StockDate'].dt.to_period('M')

# Group by the year and month, and calculate the average for the desired column
monthly_average = []
stocks['monthly_average'] = monthly_average = stocks.groupby(['organisation','year_monthstocks'])['Close'].transform('mean')
# stocks = monthly_average = stocks.groupby('year_monthstocks')['Close'].mean()

# Display the result
print(monthly_average)

0           6.622500
1         107.137497
2         122.079999
3         221.800003
4         211.840004
             ...    
310117     37.333191
310118     25.381500
310119     64.462938
310120      6.538372
310121      4.814685
Name: Close, Length: 310122, dtype: float64


C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_2056\284934890.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stocks['year_monthstocks'] = stocks['StockDate'].dt.to_period('M')


In [19]:
merged_data_stocks = pd.merge(stocks, breaches, on='organisation', how='left')

In [20]:
#remove n/a values

merged_data_stocks = merged_data_stocks.dropna(subset=['organisation','records lost']) 

In [21]:
distinct_values = merged_data_stocks['organisation'].unique()

In [22]:
print(distinct_values)

['peloton' 'amazon' 'apple' 'target' 'spotify' 'starbucks' 'nvidia'
 'adobe' 'toyota' 'uber' 'microsoft' 'ubisoft' 'nintendo' 'twitter']


In [23]:
print(merged_data_stocks)

                       StockDate       Close       Volume organisation  \
0      2025-07-03 04:00:00+00:00    6.640000    4209664.0      peloton   
3      2025-07-03 04:00:00+00:00  223.410004   29295154.0       amazon   
4      2025-07-03 04:00:00+00:00  213.550003   34697317.0        apple   
6      2025-07-03 04:00:00+00:00  104.059998    3535290.0       target   
8      2025-07-03 04:00:00+00:00  725.049988    1071771.0      spotify   
...                          ...         ...          ...          ...   
318171 2000-01-03 05:00:00+00:00    2.437765   24232000.0    starbucks   
318178 2000-01-03 05:00:00+00:00    4.468750  322352000.0       amazon   
318181 2000-01-03 05:00:00+00:00    3.990652     134500.0     nintendo   
318182 2000-01-03 05:00:00+00:00    3.990652     134500.0     nintendo   
318188 2000-01-03 05:00:00+00:00   72.034958      21200.0       toyota   

           Industry_Tag Country stock_month_year year_monthstocks  \
0               fitness     usa        Jul

In [24]:
#removing columns that had distinct values so can select distinct rows without repeats

merged_data_stocks =merged_data_stocks.drop(columns=["StockDate","Close","Volume","BreachDate","year_monthstocks"])

In [25]:
matching_dates = merged_data_stocks[merged_data_stocks['breach_month_year'] == merged_data_stocks['stock_month_year']].drop_duplicates()

In [26]:
print(matching_dates)

       organisation     Industry_Tag Country stock_month_year  \
68131       peloton          fitness     usa         May 2021   
70650        nvidia       technology     usa       March 2021   
74432       spotify            music     usa    December 2020   
84603      nintendo           gaming   japan       April 2020   
88249     microsoft       technology     usa     January 2020   
99414        toyota       automotive   japan       April 2019   
104875       amazon       e-commerce     usa    November 2018   
169904       target           retail     usa    December 2013   
171962        adobe       technology     usa     October 2013   
175187      ubisoft           gaming  france        July 2013   
175201        apple       technology     usa        July 2013   
176308     nintendo           gaming   japan        June 2013   
231940    starbucks  food & beverage     usa    November 2008   

        monthly_average records lost year        sector          method  \
68131         